In [15]:
import csv
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift


In [16]:
# Load data from the train data csv file
Feature = []
Label = []
with open('data/train/train.csv', 'r') as f:
    reader = csv.reader(f)
    train_data = list(reader)
    # remove the first row
    train_data = train_data[1:]
    for row in train_data:
        Feature.append(np.array([float(x) for x in row[1:-1]]))
        Label.append(row[-1])  
Features = np.array(Feature)
Labels = np.array(Label)

In [17]:
# Load data from the test data csv file
test_features = []
with open('data/test/test.csv', 'r') as f:
    reader = csv.reader(f)
    test_data = list(reader)
    # remove the first row
    test_data = test_data[1:]
    for row in test_data:
        test_features.append(np.array([float(x) for x in row[1:]]))
test_features = np.array(test_features)

In [18]:
pca = PCA(n_components=415)
pca.fit(Features)
Features = pca.transform(Features)
test_features = pca.transform(test_features)
lda = LinearDiscriminantAnalysis()
lda.fit(Features, Labels)
Features = lda.transform(Features)
test_features = lda.transform(test_features)

In [19]:
# implementing mean shift clustering algorithm
mean_shift = MeanShift(bandwidth=3.8)
cluster_labels = mean_shift.fit_predict(Features)
new_features = np.expand_dims(cluster_labels, axis=1)
Features = np.concatenate((Features, new_features), axis=1)

In [20]:
# # implementing k fold cross validation 
# # k = 10
# # skf = StratifiedKFold(n_splits=k, shuffle=True)
# # skf.get_n_splits(Features, Labels)
# # distance = 10
# # best_accuracy = 0
# # best_bandwidth = 0
# # for i, (train_index, test_index) in enumerate(skf.split(Features, Labels)):
# #     X_train = Features[train_index]
# #     X_test = Features[test_index]
# #     y_train = Labels[train_index]
# #     y_test = Labels[test_index]
# #     mean_shift = MeanShift(bandwidth=distance)
# #     cluster_labels = mean_shift.fit_predict(X_train)
# #     new_features = np.expand_dims(cluster_labels, axis=1)
# #     X_train = np.concatenate((X_train, new_features), axis=1)
# #     cluster_labels = mean_shift.predict(X_test)
# #     new_features = np.expand_dims(cluster_labels, axis=1)
# #     X_test = np.concatenate((X_test, new_features), axis=1)
# #     # implementing logistic regression
# #     logistic_regression = LogisticRegression(max_iter=1000)
# #     logistic_regression.fit(X_train, y_train)
# #     y_pred = logistic_regression.predict(X_test)
# #     print("Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred))
# #     accuracy = accuracy_score(y_test, y_pred)
# #     if accuracy > best_accuracy:
# #         best_accuracy = accuracy
# #         best_bandwidth = distance
# #     distance -= 0.2

# # implementing mean shift clustering algorithm with validation
# bandwidth = 4
# best_accuracy = 0
# best_bandwidth = 0
# while (bandwidth>0):
#     X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, random_state=42, stratify=Labels)
#     mean_shift = MeanShift(bandwidth=bandwidth)
#     cluster_labels = mean_shift.fit_predict(X_train)
#     new_features = np.expand_dims(cluster_labels, axis=1)
#     X_train = np.concatenate((X_train, new_features), axis=1)
#     cluster_labels = mean_shift.predict(X_test)
#     new_features = np.expand_dims(cluster_labels, axis=1)
#     X_test = np.concatenate((X_test, new_features), axis=1)
#     # implementing logistic regression
#     logistic_regression = LogisticRegression(max_iter=1000)
#     logistic_regression.fit(X_train, y_train)
#     y_pred = logistic_regression.predict(X_test)
#     print("Logistic Regression Accuracy: ", accuracy_score(y_test, y_pred))
#     accuracy = accuracy_score(y_test, y_pred)
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         best_bandwidth = bandwidth
#     bandwidth -= 0.2

In [21]:
# print("Best Accuracy: ", best_accuracy)
# print("Best Bandwidth: ", best_bandwidth)

In [22]:
# implement logistic regression
logreg = LogisticRegression(max_iter=10000)
logreg.fit(Features, Labels)
test_cluster_labels = mean_shift.predict(test_features)
new_test_features = np.expand_dims(test_cluster_labels, axis=1)
test_features = np.concatenate((test_features, new_test_features), axis=1)
test_pred = logreg.predict(test_features)

In [23]:
with open('data/test/submission_59_results.csv','w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'Category'])
    for i in range(len(test_pred)):
        writer.writerow([i, test_pred[i]])